# Задача

ДЗ делаем по бд orders

В качестве ДЗ сделаем карту поведения пользователей.

Мы обсуждали, что всех пользователей можно разделить, к примеру, на 
- !New (совершили только 1 покупку), 
- Regular (совершили 2 или более на сумму не более стольки-то), 
- Vip (совершили дорогие покупки и достаточно часто), 
- ! Lost (раньше покупали хотя бы раз и с даты последней покупки прошло больше 3 месяцев).

Вся база должна войти в эти гурппы (т.е. каждый пользователь должен попадать только в одну из этих групп).

Задача:
1. Уточнить критерии групп New,Regular,Vip,Lost
2. По состоянию на 1.01.2017 понимаем, кто попадает в какую группу, подсчитываем кол-во пользователей в каждой.
3. По состоянию на 1.02.2017 понимаем, кто вышел из каждой из групп, а кто вошел.
4. Аналогично смотрим состояние на 1.03.2017, понимаем кто вышел из каждой из групп, а кто вошел.
5. В итоге делаем вывод, какая группа уменьшается, какая увеличивается и продумываем, в чем может быть причина.

Присылайте отчет в pdf
*/

# Подготовка

## Подключение библиотек

In [1]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [2]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [30]:
def show_tables():
    return pd.read_sql("""SHOW TABLES FROM db_analyt_loc;""", con=mydb)
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b
3,time_user_ym_s
4,users_for_merg


1. Уточнить критерии групп New,Regular,Vip,Lost


## 2. По состоянию на 1.01.2017 понимаем, кто попадает в какую группу, подсчитываем кол-во пользователей в каждой.

In [53]:
current_db = 'orders_20190822'

In [64]:
date = '\'2017-01-01\''

In [79]:
%%time
df_new = pd.read_sql(
    f"""
    SELECT 
      user_id, 
      'New' AS group_,
      DATE('2017-01-01') AS date
    FROM 
    (SELECT * FROM orders_20190822 WHERE o_date < '2017-01-01') AS o2
    GROUP BY user_id
    HAVING COUNT(id_o) = 1
    """, con=mydb)
df_new

Wall time: 10.7 s


,user_id,group_,date
0,1105609,New,2017-01-01
1,982696,New,2017-01-01
2,1105614,New,2017-01-01
3,1105617,New,2017-01-01
4,1105621,New,2017-01-01
...,...,...,...
334082,2838382,New,2017-01-01
334083,2838390,New,2017-01-01
334084,2838399,New,2017-01-01
334085,2838406,New,2017-01-01


In [66]:
%%time
df_lost = pd.read_sql(
    f"""
    SELECT
      user_id, 
      'Lost' AS group_,
      DATE({date}) AS date
    FROM (
    SELECT 
        user_id,
        TIMESTAMPDIFF(MONTH,MAX(o_date),DATE('2017-12-31')) AS period_from_last_ord
    FROM (SELECT * FROM {current_db} WHERE o_date < {date}) AS o2
    GROUP BY user_id
    HAVING COUNT(id_o >= 1)) AS t
    WHERE period_from_last_ord >= 3;
    """, con=mydb)
df_new.shape

Wall time: 13.1 s


(334087, 3)

In [68]:
%%time
df_vip = pd.read_sql(
    f"""
    SELECT
      t2.user_id, 
      'Vip' AS group_,
      DATE({date}) AS date
    FROM (
    SELECT 
      user_id,
      CEILING(COUNT(o.id_o)/TIMESTAMPDIFF(MONTH,MIN(o.o_date),MAX(o.o_date))) AS freq,
      ROUND(AVG(o.price),2) as avg_price
    FROM {current_db} o
    GROUP BY o.user_id) AS t2
    WHERE t2.freq >2 AND t2.avg_price > 1000;
    """, con=mydb)
df_new.shape

Wall time: 5.35 s


(334087, 3)

In [69]:
%%time
df_regular = pd.read_sql(
    f"""
    SELECT
      t2.user_id, 
      'Regular' AS group_,
      DATE({date}) AS date
    FROM (
    SELECT 
      user_id,
      CEILING(COUNT(o.id_o)/TIMESTAMPDIFF(MONTH,MIN(o.o_date),MAX(o.o_date))) AS freq,
      ROUND(AVG(o.price),2) as avg_price
    FROM {current_db} o
    GROUP BY o.user_id) AS t2
    WHERE t2.freq >2 AND t2.avg_price <= 1000;
    """, con=mydb)
df_new.shape

Wall time: 4.86 s


(334087, 3)

In [71]:
%%time
pd.read_sql(
    f"""
    SELECT * from {current_db};
    """, con=mydb)
df_new.shape

Wall time: 54.9 s


(334087, 3)

In [70]:
sum(df_new.shape[0],df_lost.shape[0])

334087

### 2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)

### 3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).

### 4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

In [ ]:
%%time
mydb.cursor().execute(
    """
    CREATE TABLE time_user_ym_s
    SELECT
      user_id,
      DATE_FORMAT(o_date, '%y%m') AS ym,
      SUM(price) AS s
    FROM orders_20190822 o
    GROUP BY user_id, DATE_FORMAT(o_date, '%y%m');
    """
)
mydb.commit()

In [ ]:
pd.read_sql(f"SELECT * FROM time_user_ym_s", con=mydb)

# Закрытие сессии

In [ ]:
mydb.close()